In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict
import random
import requests
from utils.match_prediction.champions import Champion
from utils.match_prediction import RAW_PRO_GAMES_FILE
from IPython.display import display, clear_output
import ipywidgets as widgets
import re
import webbrowser

# Load the dataset
df = pd.read_parquet(RAW_PRO_GAMES_FILE)

# Add relabel column if it doesn't exist
if "team_100_win_relabel" not in df.columns:
    df["team_100_win_relabel"] = -1
df["team_100_win_relabel"] = df["team_100_win_relabel"].astype(
    float
)  # Explicitly cast to float


# Create champion id to name mapping
CHAMPION_ID_TO_NAME: Dict[int, str] = {
    champion.id: champion.display_name for champion in Champion
}


def format_champion_matchup(champion_ids: List[int]) -> str:
    """Format champion matchups in a visually appealing way"""
    blue_team = champion_ids[:5]
    red_team = champion_ids[5:]

    formatted_output = []
    for i in range(5):
        blue_name = CHAMPION_ID_TO_NAME.get(blue_team[i], f"Unknown({blue_team[i]})")
        red_name = CHAMPION_ID_TO_NAME.get(red_team[i], f"Unknown({red_team[i]})")
        # Pad names to align nicely
        formatted_output.append(f"{blue_name:<15} vs {red_name}")

    return "\n".join(formatted_output)


# TODO: refactor this into a function in utils
def get_model_prediction(champion_ids: List[int], patch: str) -> dict:
    """Get model prediction from the API"""
    url = "http://localhost:8000/predict-in-depth"
    payload = {
        "champion_ids": champion_ids,
        "numerical_elo": 0,  # Pro play elo
        "patch": patch,
    }
    headers = {"X-API-Key": "example_token"}

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error getting prediction: {e}")
        return None


def get_unrelabeled_game(even_only: bool = True, min_error: float = 0.8) -> pd.Series:
    """Get the most recent unrelabeled game based on golgg_id, considering only even or odd indices.

    Args:
        even_only: If True, only consider games with even indices. If False, only odd indices.
        min_error: Minimum model error threshold to consider for relabeling.

    Returns:
        pd.Series: A single game row, or None if no games are available
    """
    # Get unrelabeled games with high model error and sort by golgg_id in descending order
    unrelabeled_games = (
        df[(df["team_100_win_relabel"] == -1) & (df["model_error"] >= min_error)]
        .sort_values("golgg_id", ascending=False)
        .reset_index(drop=True)
    )

    if len(unrelabeled_games) == 0:
        print("No high-error games left to relabel!")
        return None

    # Filter for even or odd golgg_ids
    filtered_games = unrelabeled_games[
        unrelabeled_games["golgg_id"] % 2 == (0 if even_only else 1)
    ]

    if len(filtered_games) == 0:
        print(
            f"No {'even' if even_only else 'odd'} indexed high-error games left to relabel!"
        )
        return None

    # Return the first game (highest golgg_id)
    return filtered_games.iloc[0]


class RelabelInterface:
    def __init__(self):
        # Create widgets
        self.output = widgets.Output()
        self.label_input = widgets.Text(
            description="Label:",
            placeholder="Enter value between 0 and 1",
            layout=widgets.Layout(width="300px", height="40px"),
            style={"description_width": "initial", "font_size": "16px"},
            description="Label:",
            placeholder="Enter value between 0 and 1",
            layout=widgets.Layout(width="300px", height="40px"),
            style={"description_width": "initial", "font_size": "16px"},
        )
        self.refresh_button = widgets.Button(
            description="Skip/Refresh",
            button_style="info",
            layout=widgets.Layout(width="150px", height="40px"),
            style={"font_size": "16px"},
            description="Skip/Refresh",
            button_style="info",
            layout=widgets.Layout(width="150px", height="40px"),
            style={"font_size": "16px"},
        )
        self.relabel_button = widgets.Button(
            description="Relabel",
            button_style="success",
            layout=widgets.Layout(width="150px", height="40px"),
            style={"font_size": "16px"},
        )
        self.status_label = widgets.HTML(
            value="<b>Status:</b> Ready to relabel",
            layout=widgets.Layout(width="300px", height="40px"),
            style={"font_size": "16px"},
            description="Relabel",
            button_style="success",
            layout=widgets.Layout(width="150px", height="40px"),
            style={"font_size": "16px"},
        )
        self.status_label = widgets.HTML(
            value="<b>Status:</b> Ready to relabel",
            layout=widgets.Layout(width="300px", height="40px"),
            style={"font_size": "16px"},
        )

        # Set up layout
        self.container = widgets.VBox(
            [
                self.output,
                widgets.HBox(
                    [self.label_input, self.refresh_button, self.relabel_button]
                ),
                self.status_label,
            ]
        )

        # Add button callbacks
        self.refresh_button.on_click(self.refresh_game)
        self.relabel_button.on_click(self.relabel_current_game)

        # Initialize state
        self.current_game_id = None
        self.current_golgg_id = None  # Add this line
        self.display_new_game()

    def display_new_game(self):
        """Display a new random game"""
        with self.output:
            clear_output()
            game = get_unrelabeled_game()
            if game is None:
                print("All games have been relabeled!")
                return

            self.current_game_id = game.name
            self.current_golgg_id = game.golgg_id  # Add this line

            # Calculate progress
            total = len(df)
            relabeled = (df["team_100_win_relabel"] != -1).sum()
            progress_pct = relabeled / total

            # Automatically open the gol.gg page
            webbrowser.open(f"https://gol.gg/game/stats/{game.golgg_id}/page-game/")

            # Use HTML for styled output
            html_output = f"""
            <div style="background-color: #1a1a1a; color: #ffffff; padding: 20px; font-size: 14px; font-family: monospace;">
                <div style="font-size: 16px; margin-bottom: 20px;">
                    <div style="color: #888888;">Game ID: 
                        <a href="https://gol.gg/game/stats/{game.golgg_id}/page-game/" 
                          target="_blank" 
                          style="color: #4d9fff; text-decoration: none; border-bottom: 1px dotted #4d9fff;">
                            {game.golgg_id}
                        </a>
                    </div>
                    <div style="margin-top: 10px;">
                        <div>Patch: {game.gameVersionMajorPatch}.{str(game.gameVersionMinorPatch).zfill(2)}</div>
                        <div>Duration: {game.gameDuration//60}:{game.gameDuration%60:02d}</div>
                        <div>Tournament: {game.tournamentName}</div>
                        <div>Teams: {game.blueTeamName} vs {game.redTeamName}</div>
                        <div>Actual Result: {'Blue' if game.team_100_win else 'Red'} Team Won</div>
                    </div>
                </div>
            """

            # Get model prediction
            patch = f"{game.gameVersionMajorPatch}.{str(game.gameVersionMinorPatch).zfill(2)}"
            prediction = get_model_prediction(game.champion_ids.tolist(), patch)

            if prediction:
                win_prob = prediction["win_probability"]
                self.label_input.value = f"{win_prob:.3f}"

                # Add prediction section
                html_output += f"""
                <div style="margin-bottom: 20px;">
                    <div style="font-size: 16px; color: #888888;">Model Prediction</div>
                    <div style="display: flex; justify-content: space-between; margin-top: 5px;">
                        <span style="color: #4d9fff;">{win_prob:.1%} Blue Side</span>
                        <span style="color: #ff4d4d;">{(1-win_prob):.1%} Red Side</span>
                    </div>
                </div>
                """

                # Start lane analysis table
                html_output += """
                <div style="font-size: 16px; color: #888888; margin-bottom: 10px;">Lane Analysis</div>
                <table style="width: 100%; border-collapse: collapse; font-size: 14px;">
                    <tr style="border-bottom: 1px solid #333333;">
                        <th style="text-align: left; padding: 8px; color: #4d9fff;">BLUE SIDE</th>
                        <th style="text-align: right; padding: 8px; color: #888888;">IMPACT</th>
                        <th style="text-align: right; padding: 8px; color: #888888;">G@15</th>
                        <th style="text-align: center; padding: 8px; color: #888888;">|</th>
                        <th style="text-align: left; padding: 8px; color: #888888;">G@15</th>
                        <th style="text-align: left; padding: 8px; color: #888888;">IMPACT</th>
                        <th style="text-align: right; padding: 8px; color: #ff4d4d;">RED SIDE</th>
                    </tr>
                """

                roles = ["TOP", "JNG", "MID", "BOT", "SUP"]
                for i in range(5):
                    blue_champ = CHAMPION_ID_TO_NAME.get(game.champion_ids[i])
                    red_champ = CHAMPION_ID_TO_NAME.get(game.champion_ids[i + 5])
                    blue_impact = prediction["champion_impact"][i]
                    red_impact = prediction["champion_impact"][i + 5]
                    gold_diff = prediction["gold_diff_15min"][i]

                    # Format gold differences
                    blue_gold = f"{abs(gold_diff):,.0f}" if gold_diff > 0 else ""
                    red_gold = f"{abs(gold_diff):,.0f}" if gold_diff < 0 else ""

                    # Format impact scores with colors
                    blue_impact_color = (
                        "#00cc00"
                        if blue_impact > 0
                        else "#cc0000" if blue_impact < 0 else "#888888"
                    )
                    red_impact_color = (
                        "#00cc00"
                        if red_impact > 0
                        else "#cc0000" if red_impact < 0 else "#888888"
                    )
                    blue_impact_str = f"{blue_impact:+.1%}" if blue_impact != 0 else ""
                    red_impact_str = f"{red_impact:+.1%}" if red_impact != 0 else ""

                    html_output += f"""
                    <tr style="border-bottom: 1px solid #333333;">
                        <td style="text-align: left; padding: 8px; color: #4d9fff;">{blue_champ}</td>
                        <td style="text-align: right; padding: 8px; color: {blue_impact_color};">{blue_impact_str}</td>
                        <td style="text-align: right; padding: 8px; color: #00cc00;">{blue_gold}</td>
                        <td style="text-align: center; padding: 8px; color: #888888;">|</td>
                        <td style="text-align: left; padding: 8px; color: #00cc00;">{red_gold}</td>
                        <td style="text-align: left; padding: 8px; color: {red_impact_color};">{red_impact_str}</td>
                        <td style="text-align: right; padding: 8px; color: #ff4d4d;">{red_champ}</td>
                    </tr>
                    """

                html_output += """
                </table>
                </div>
                """

                html_output += f"""
                        <div style="margin-top: 20px; padding-top: 20px; border-top: 1px solid #333333;">
                            <div style="font-size: 16px; color: #888888;">Progress</div>
                            <div style="display: flex; justify-content: space-between; align-items: center; margin-top: 10px;">
                                <div>
                                    <span style="color: #00cc00;">{relabeled}</span>
                                    <span style="color: #888888;">/</span>
                                    <span style="color: #000000;">{total}</span>
                                    <span style="color: #888888;"> games relabeled</span>
                                </div>
                                <div style="color: #00cc00;">
                                    {progress_pct:.1%}
                                </div>
                            </div>
                            <div style="width: 100%; height: 4px; background-color: #333333; margin-top: 8px; border-radius: 2px;">
                                <div style="width: {progress_pct:.1%}; height: 100%; background-color: #00cc00; border-radius: 2px;"></div>
                            </div>
                        </div>
                    </div>
                """

                from IPython.display import HTML

                display(HTML(html_output))

    def refresh_game(self, _):
        """Show a new game"""
        self.status_label.value = "<b>Status:</b> Loading new game..."
        self.display_new_game()
        self.status_label.value = "<b>Status:</b> Ready to relabel"

    def relabel_current_game(self, _):
        """Relabel the current game"""
        if self.current_golgg_id is None:  # Change this line
            self.status_label.value = "<b>Status:</b> No game selected"
            return

        # Validate input
        try:
            # accept french notation
            cleaned_input = self.label_input.value.replace(",", ".")
            new_label = float(cleaned_input)
            if not 0 <= new_label <= 1:
                raise ValueError("Label must be between 0 and 1")
        except ValueError as e:
            self.status_label.value = f"<b>Status:</b> Invalid input - {str(e)}"
            return

        # Update the dataset using golgg_id
        df.loc[df["golgg_id"] == self.current_golgg_id, "team_100_win_relabel"] = (
            new_label  # Change this line
        )
        df.to_parquet(RAW_PRO_GAMES_FILE)

        # Show success and load new game
        self.status_label.value = f"<b>Status:</b> Game {self.current_golgg_id} relabeled with {new_label:.3f}"
        self.display_new_game()

    def show(self):
        """Display the interface"""
        display(self.container)


# Create and show the interface
def start_relabeling():
    interface = RelabelInterface()
    interface.show()


# Show progress
def show_progress():
    total = len(df)
    relabeled = (df["team_100_win_relabel"] != -1).sum()
    print(f"Progress: {relabeled}/{total} games relabeled ({relabeled/total:.1%})")

In [ ]:
start_relabeling()